<a href="https://colab.research.google.com/github/gustavoamora/case-ton/blob/main/case_ton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bibliotecas

## Instalações

In [ ]:
# Instalações que podem ser necessárias, direto com o pip

!pip install scipy==1.11.4 --quiet
!pip install mlxtend==0.22.0 --quiet
!pip install plotly==5.15.0 --quiet
# !pip install --upgrade ipykernel IPython --quiet

## Import

In [ ]:
#Libs para manipulação:
import pandas as pd
import numpy as np
import sqlite3
from collections import defaultdict

# Libs para gráficos:
import plotly.express as px # Lib. para gráficos
import plotly.graph_objs as go # Lib. para gráficos

# Libs. para estatísticas:
from scipy import stats # Lib. para algoritmos estatísticos
from scipy.stats import ttest_ind
from numpy.polynomial import Polynomial
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Pré-processamento

Tratar os dados das tabelas para as análises propostas.

### Tabela 'cases'

Chamados recebidos pelo time de Relacionamento com o Cliente.

In [ ]:
### case.csv:
### Transformar o case.csv em um dataframe

df_case = pd.read_csv('case.csv')

In [ ]:
### Entender o shape do df e os nomes das colunas

df_case.shape, df_case.keys()

((126989, 9),
 Index(['Unnamed: 0', 'accountid', 'date_ref', 'channelid', 'waitingtime',
        'missed', 'pesquisa_de_satisfa_o__c', 'assunto', 'Id'],
       dtype='object'))

In [ ]:
### Overview dos primeiros registros do df

df_case.head()

,Unnamed: 0,accountid,date_ref,channelid,waitingtime,missed,pesquisa_de_satisfa_o__c,assunto,Id
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0013j00002z0CeEAAU,2020-07-31,2.0,15.0,False,NaN,Aplicativo:Dúvidas funcionalidades App:Primeir...,0013j00002z0CeEAAU


In [ ]:
### Percebe-se que a coluna ['Unnamed: 0'] = index. Como o df já possui um index, será excluída.

df_case.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
### Verificar a exclusão da coluna anterior:

df_case.shape, df_case.keys()

((126989, 8),
 Index(['accountid', 'date_ref', 'channelid', 'waitingtime', 'missed',
        'pesquisa_de_satisfa_o__c', 'assunto', 'Id'],
       dtype='object'))

In [ ]:
### Verificar os tipos dos dados do df

df_case.dtypes

accountid                    object
date_ref                     object
channelid                   float64
waitingtime                 float64
missed                       object
pesquisa_de_satisfa_o__c     object
assunto                      object
Id                           object
dtype: object

In [ ]:
### A feature 'date_ref' (data do chamado) está como tipo = object e o ideal para as análise deve ser do tipo 'datetime'. Por isso, vamos convertê-la:

df_case['date_ref'] = pd.to_datetime(df_case['date_ref'])

In [ ]:
### Verificar a alteração acima:

df_case.dtypes

accountid                           object
date_ref                    datetime64[ns]
channelid                          float64
waitingtime                        float64
missed                              object
pesquisa_de_satisfa_o__c            object
assunto                             object
Id                                  object
dtype: object

In [ ]:
### Verificar a quantidade de registros totalmente nulos:

reg_null = df_case.isnull().all(axis=1).sum()
print("Total de registros nulos:", reg_null)

Total de registros nulos: 49500


In [ ]:
### Excluir os registros totalmente nulos.

df_case.dropna(how='all', inplace=True)

In [ ]:
### Verificar a exclusão e o novo shape do df:

df_case.shape, df_case.dtypes

((77489, 8),
 accountid                           object
 date_ref                    datetime64[ns]
 channelid                          float64
 waitingtime                        float64
 missed                              object
 pesquisa_de_satisfa_o__c            object
 assunto                             object
 Id                                  object
 dtype: object)

In [ ]:
### Verificar a quantidade de nulls por feature:

feat_null = df_case.isnull().sum()
print("Valores nulos por feature:\n", feat_null)

Valores nulos por feature:
 accountid                       0
date_ref                        0
channelid                       0
waitingtime                     0
missed                          0
pesquisa_de_satisfa_o__c    65904
assunto                         0
Id                              0
dtype: int64


### Tabela 'creds'

Clientes credenciados.

In [ ]:
### cred.csv:
### Transformar o cred.csv em um dataframe

df_cred = pd.read_csv('cred.csv')

In [ ]:
### Entender o shape do df e os nomes das colunas

df_cred.shape, df_cred.keys()

((126989, 6),
 Index(['Unnamed: 0', 'cred_date', 'shipping_address_city',
        'shipping_address_state', 'max_machine', 'accountid'],
       dtype='object'))

In [ ]:
### Overview dos primeiros registros do df

df_cred.head()

,Unnamed: 0,cred_date,shipping_address_city,shipping_address_state,max_machine,accountid
0,0,2020-04-18,Feira de Santana,BA,T1,NaN
1,1,2020-10-16,Bacuri,MA,T1,NaN
2,2,2020-09-01,Bernardo Sayão,TO,T1,NaN
3,3,2020-08-29,Rio de Janeiro,RJ,T3,NaN
4,4,2020-07-28,São Gonçalo,RJ,T3,0013j00002z0CeEAAU


In [ ]:
### Percebe-se que a coluna ['Unnamed: 0'] = index. Como o df já possui esse campo, iremos retirá-la.

df_cred.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
### Verificar a exclusão da coluna anterior:

df_cred.shape, df_cred.keys()

((126989, 5),
 Index(['cred_date', 'shipping_address_city', 'shipping_address_state',
        'max_machine', 'accountid'],
       dtype='object'))

In [ ]:
### Verificar os tipos dos dados do df

df_cred.dtypes

cred_date                 object
shipping_address_city     object
shipping_address_state    object
max_machine               object
accountid                 object
dtype: object

In [ ]:
### Constatamos que a feature 'cred_date' (Data de credenciamento) está como tipo = object e o ideal para as análise deveria ser tipo = 'datetime'. Por isso, vamos convertê-la:

df_cred['cred_date'] = pd.to_datetime(df_cred['cred_date'])

In [ ]:
### Verificar a alteração acima:

df_cred.dtypes

cred_date                 datetime64[ns]
shipping_address_city             object
shipping_address_state            object
max_machine                       object
accountid                         object
dtype: object

In [ ]:
### Verificar a quantidade de registros totalmente nulos:

reg_null = df_cred.isnull().all(axis=1).sum()
print("Total de registros nulos:", reg_null)

Total de registros nulos: 0


In [ ]:
### Verificar a quantidade de nulls por feature:

feat_null = df_cred.isnull().sum()
print("Valores nulos por feature:\n", feat_null)

Valores nulos por feature:
 cred_date                     0
shipping_address_city         0
shipping_address_state        0
max_machine                   0
accountid                 49500
dtype: int64


# Q1

Foi implementada no dia 1 de agosto uma funcionalidade nova no site do TON que
acreditam que irá diminuir a quantidade de chamados recebidos relacionados ao nosso produto. Avalie se a implementação teve um impacto estatisticamente relevante.

## Desenvolvimento

In [ ]:
# Invervalo de datas do df_case:
data_min_case = df_case['date_ref'].min()
data_max_case = df_case['date_ref'].max()

# Invervalo de datas do df_cred:
data_min_cred = df_cred['cred_date'].min()
data_max_cred = df_cred['cred_date'].max()
print(f'Intervalo de datas do df_case está entre {data_min_case} e {data_max_case}\nIntervalo de datas do df_cred está entre {data_min_cred} e {data_max_cred}')

Intervalo de datas do df_case está entre 2020-02-23 00:00:00 e 2020-10-29 00:00:00
Intervalo de datas do df_cred está entre 2019-07-11 00:00:00 e 2020-10-30 00:00:00


In [ ]:
# Contagem de chamados por dia:
chamados_por_dia = df_case.groupby(df_case['date_ref'].dt.date).size().reset_index(name='quantidade_de_chamados')

# Definindo corte de data da implementação
cutoff_date = pd.to_datetime('2020-08-01').date()

# Add. coluna auxiliar para viz. gráfica.
chamados_por_dia['periodo'] = chamados_por_dia['date_ref'].apply(lambda x: 'Pré implantação da funcionalidade' if x < cutoff_date else 'Pós implantação da funcionalidade')

In [ ]:
# Contagem de clientes credenciados por dia:
n_clientes = df_cred.groupby(df_cred['cred_date'].dt.date).size().reset_index(name='n_clientes_por_dia')

# Número de clientes acumulado por dia:
n_clientes['n_clientes_acc'] = n_clientes['n_clientes_por_dia'].cumsum()

# Add. coluna auxiliar para viz. gráfica.
n_clientes['periodo'] = n_clientes['cred_date'].apply(lambda x: 'Pré implantação da funcionalidade' if x < cutoff_date else 'Pós implantação da funcionalidade')

In [ ]:
# Viz. gráfica da time series com o número de clientes na base, com separação da data pré e pós implementação da nova funcionalidade e sem considerar churn.

fig = px.line(n_clientes, x='cred_date', y='n_clientes_acc', color='periodo', title='Quantidade de clientes na base ao longo do tempo')
fig.show()

In [ ]:
# Viz. gráfica da time series com as quantidades de chamados por dia, com separação da data pré e pós implementação da nova funcionalidade.

fig = px.line(chamados_por_dia, x='date_ref', y='quantidade_de_chamados', color='periodo', title='Número de chamados por dia')
fig.show()

O número de clientes credenciados na base pode influenciar significativamente a análise da quantidade de chamados abertos nos períodos pré e pós a implementação da nova funcionalidade. Ou seja, se a base de clientes cresceu ou diminuiu na comparação dos períodos isso pode afetar a quantidade de chamados independentemente da nova funcionalidade.

Para contornar esse problema, a abordagem utilizada será a de calcular uma taxa de chamados por cliente.

Obs: o número de clientes credenciados por si só não é uma medida totalmente confiável pois, por exemplo, não estamos levando em consideração nesta análise o churn, que pode diminuir o número de clientes ativos. Outro exemplo é que não estamos considerando o volume de transações dos clientes ativos, que, logicamente, quanto mais utilizam o produto maior a probabilidade de se deparar com problemas e assim abrir um chamado.

In [ ]:
# Sincronizar as features de 'date' nos dois dfs:
chamados_por_dia.rename(columns={'date_ref': 'data'}, inplace=True)
n_clientes.rename(columns={'cred_date': 'data'}, inplace=True)

# Merge dos dfs
df_combinado = pd.merge(chamados_por_dia, n_clientes, on=['data', 'periodo'])

In [ ]:
df_combinado.head()

,data,quantidade_de_chamados,periodo,n_clientes_por_dia,n_clientes_acc
0,2020-02-23,1,Pré implantação da funcionalidade,1,7981
1,2020-02-25,1,Pré implantação da funcionalidade,1,7982
2,2020-02-26,22,Pré implantação da funcionalidade,35,8017
3,2020-02-27,31,Pré implantação da funcionalidade,53,8070
4,2020-02-28,27,Pré implantação da funcionalidade,55,8125


In [ ]:
# Excluir a feature 'n_clientes_por_dia' e reorganizando o df_combinado:
ordem = ['data', 'n_clientes_acc', 'quantidade_de_chamados', 'periodo']
df_comb = df_combinado[ordem]
df_comb.head()

,data,n_clientes_acc,quantidade_de_chamados,periodo
0,2020-02-23,7981,1,Pré implantação da funcionalidade
1,2020-02-25,7982,1,Pré implantação da funcionalidade
2,2020-02-26,8017,22,Pré implantação da funcionalidade
3,2020-02-27,8070,31,Pré implantação da funcionalidade
4,2020-02-28,8125,27,Pré implantação da funcionalidade


In [ ]:
df_comb = df_comb.copy()

# Add. coluna com a taxa (quantidade de chamados / número de clientes).
df_comb['taxa'] = df_comb['quantidade_de_chamados'] / df_comb['n_clientes_acc']
df_comb.head()

,data,n_clientes_acc,quantidade_de_chamados,periodo,taxa
0,2020-02-23,7981,1,Pré implantação da funcionalidade,0.000125
1,2020-02-25,7982,1,Pré implantação da funcionalidade,0.000125
2,2020-02-26,8017,22,Pré implantação da funcionalidade,0.002744
3,2020-02-27,8070,31,Pré implantação da funcionalidade,0.003841
4,2020-02-28,8125,27,Pré implantação da funcionalidade,0.003323


In [ ]:
# Alterar feature data para tipo datetime
df_comb['data'] = pd.to_datetime(df_comb['data'])

In [ ]:
import plotly.express as px
import plotly.graph_objs as go
from numpy.polynomial import Polynomial

# Viz. gráfica da time series com as taxas por dia, com separação da data pré e pós implementação da nova funcionalidade.
fig = px.line(df_comb, x='data', y='taxa', color='periodo', title='Taxa de chamados por dia')

# Adicionar linhas de tendência para cada grupo
for periodo in df_comb['periodo'].unique():
    df_temp = df_comb[df_comb['periodo'] == periodo]
    # Converte datas para um formato numérico
    dates_numeric = pd.to_datetime(df_temp['data']).astype(int)

    # Calcular a regressão linear
    p = Polynomial.fit(dates_numeric, df_temp['taxa'], 1)

    # Gerar valores da linha de tendência
    trendline_values = p(dates_numeric)

    # Adicionar a linha de tendência ao gráfico
    fig.add_trace(go.Scatter(x=df_temp['data'], y=trendline_values, mode='lines',
                             name=f'Tendência {periodo}'))

fig.show()

Agora com a taxa é possível verificar se o resultado da implantação da nova funcionalidade foi estatisticamente relevante ou não.

In [ ]:
# Divisão dos dados em pré e pós implantação da nova funcionalidade

pre = df_comb[df_comb['data'] < '2020-08-01']
pos = df_comb[df_comb['data'] >= '2020-08-01']

In [ ]:
# Estatisticas descritivas dos dfs pré e pós

pre['taxa'].describe(), pos['taxa'].describe()

(count    155.000000
 mean       0.005945
 std        0.003685
 min        0.000103
 25%        0.002694
 50%        0.005776
 75%        0.008599
 max        0.014144
 Name: taxa, dtype: float64,
 count    90.000000
 mean      0.005642
 std       0.002514
 min       0.001356
 25%       0.003160
 50%       0.006178
 75%       0.007678
 max       0.010849
 Name: taxa, dtype: float64)

In [ ]:
# Aplicação do teste t para verificar a relevância estatística da diferença das medidas entre as duas amostras (pré e pós)
from scipy.stats import ttest_ind

t_stat, p_val = ttest_ind(pre['taxa'], pos['taxa'])
print(f'T-statistic: {t_stat}, P-value: {p_val}')

T-statistic: 0.6932837817917725, P-value: 0.4887935851053352


O teste estatístico t [(Student's t-test)](https://en.wikipedia.org/wiki/Student%27s_t-test) é usado para determinar se há uma diferença significativa entre as médias de duas amostras independentes. No caso, as amostras são os dados da taxa nos dados pre e pos implantação da funcionalidade.

Resultados:

T-statistic = 0.69. Essa estatística mede a distância entre as médias das duas amostras em termos de erro padrão. Um valor mais alto da estatística T sugere uma maior diferença entre as médias.

P-value = 0.49. O valor-p é usado para aceitar ou rejeitar a hipótese nula. A hipótese nula, neste caso, é de que as médias das duas amostras são iguais. Usualmente o intervalo de confiança corte para rejeitar é acima de 0.05, que sugere que a diferença observada nas médias das amostras é estatisticamente significativa.

No caso analisado, com um valor-p de aproximadamente 0.49, não há evidência estatística suficiente para concluir que as médias das taxas pré e pós são diferentes. Isso significa que, com base nos dados fornecidos e no nível de significância usual de 0.05, não se pode afirmar que há uma diferença significativa entre os grupos pré e pós em termos da variável 'taxa'.

Contudo, ao observamos o gráfico da série temporal da taxa e sua linha de regressão, vemos que claramente existia uma tendêNcia de subida da taxa antes da implantação da funcionalidade e uma tendência de queda na taxa pós implantação. Ainda, é possível observar que no início da série temporal, de fevereiro até maio aproximadamente, a taxa era visivelmente menor. Isso pode ser por vários motivos e aparentemente está 'puxando' o período pré implantação para uma média menor no pré-implantação.

Por isso vamos testar também os 90 dias pós implantação com os 90 dias da pré-implantação.

In [ ]:
# Divisão dos dados em aproximadamente 3 meses pré e 3 meses pós implantação da nova funcionalidade:

pre_2 = df_comb[(df_comb['data'] >= '2020-05-01') & (df_comb['data'] < '2020-08-01')]
pos_2 = df_comb[df_comb['data'] >= '2020-08-01']

In [ ]:
# Estatisticas descritivas dos dfs pré e pós (3 meses)

pre_2['taxa'].describe(), pos_2['taxa'].describe()

(count    92.000000
 mean      0.007610
 std       0.003486
 min       0.001443
 25%       0.004799
 50%       0.008026
 75%       0.010286
 max       0.014144
 Name: taxa, dtype: float64,
 count    90.000000
 mean      0.005642
 std       0.002514
 min       0.001356
 25%       0.003160
 50%       0.006178
 75%       0.007678
 max       0.010849
 Name: taxa, dtype: float64)

In [ ]:
# Aplicação do teste t para verificar a relevância estatística da diferença das medidas entre as duas amostras (pré e pós)
from scipy.stats import ttest_ind

t_stat, p_val = ttest_ind(pre_2['taxa'], pos_2['taxa'])
print(f'T-statistic: {t_stat}, P-value: {p_val}')

T-statistic: 4.3615970675942615, P-value: 2.169748353848312e-05


T-statistic = 4.36. Resultado relativamente alto, indicando uma diferença substancial entre as médias das duas amostras. Isso sugere que a diferença entre as médias não é provável que seja resultado do acaso.

P-value = 0.0000217. Este é um valor muito baixo, significativamente menor do que o usual 0.05 para significância estatística.

Os resultados indicam que a diferença nas médias das taxas entre os períodos pré e pós no novo corte (3 meses pré e 3 meses pós) é estatisticamente significativa.

## Conclusão

Ao comparar a taxa de chamados abertos por cliente nos 3 meses pós implantação da nova funcionalidade com os 3 meses anteriores a ela, pode se afirmar que houve uma queda, suportada por significativa relevância estatística.

A lógica para resolulçao do problema foi a seguinte:

1.   Calcular uma taxa de número de chamados / número de clientes para contornar o efeito da subida/queda do número de tickets criados ser influenciada pelo aumento/diminuição do número clientes.
2.   Dividir os dados em pré e pós implantação de acordo com a data de lançamento da funcionalidade.
2.   Plotar a taxa em uma série temporal, juntamente com sua regressão linear, com a divisão pré e pós implantação da nova funcionalidade (ver na próxima célula).
3.   Analisar o gráfico e entender a tendência da regressão
4.   Aplicar o t-test e analisar os resultados.
5.   Ao analisar o passo anterior, não foi constatada relevância estatística para na comparação pós implantação e pré implantação. Contudo, a série temporal pré era bem mais longa e a sua regressão no gráfico com inclinação positiva, enquanto a regressão no pós estava negativa. Visivelmente também existiam taxas bem pequenas no início da série temporal, o que levantou a suspeita de que estavam 'puxando' as médias para baixo.
6.   Para uma comparação do efeito da nova funcionalidade mais justa, separamos os dados pré em um intervalo de tempo aproximadamente igual ao que tínhamos de dados pós (3 meses)
7.   Repetimos o t-test e os resultados sugerem uma diferença consideravél entre as amostras, com queda da taxa no pós implantação da funcionalidade e com relevância estatística significativa para confirmar.

In [ ]:
# Criar gráfico
fig = px.line(df_comb, x='data', y='taxa', color='periodo', title='Taxa de chamados por dia')

# Adicionar linhas de tendência para cada grupo
for periodo in df_comb['periodo'].unique():
    df_temp = df_comb[df_comb['periodo'] == periodo]
    # Converte datas para um formato numérico
    dates_numeric = pd.to_datetime(df_temp['data']).astype(int)

    # Calcular a regressão linear
    p = Polynomial.fit(dates_numeric, df_temp['taxa'], 1)

    # Gerar valores da linha de tendência
    trendline_values = p(dates_numeric)

    # Adicionar a linha de tendência ao gráfico
    fig.add_trace(go.Scatter(x=df_temp['data'], y=trendline_values, mode='lines',
                             name=f'Tendência {periodo}'))

fig.show()

# Q2

Foi notado por alguns agentes do Relacionamento com o Cliente que clientes mais
recentes possuem um comportamento específico em relação à quantidade de
chamados realizados logo após seu credenciamento. Prepare alguma forma de
visualizar essa diferença notada pelos agentes.

## Desenvolvimento

In [ ]:
# Durante o pré processamento foi verificado que existem muitos valores nulos na feature 'accountid'.
# Sem essa feature não é possível fazer o rastreamento dos clientes e seus respectivos chamados abertos
# Sendo assim, vamos excluir todos os registros que contém valores nulos em 'accountid'.

df_cred_sem_nulos = df_cred.dropna(subset=['accountid'])
df_cred_sem_nulos.shape, df_cred_sem_nulos.isnull().sum()


((77489, 5),
 cred_date                 0
 shipping_address_city     0
 shipping_address_state    0
 max_machine               0
 accountid                 0
 dtype: int64)

In [ ]:
# União dos datasets df_case e df_cred_sem_nulos:

# Mesclar os DataFrames
df_merged = pd.merge(df_case, df_cred_sem_nulos, on='accountid')

# Calcular a diferença de tempo entre 'data_ref' e 'data_cred'
df_merged['dias_apos_venda'] = (df_merged['date_ref'] - df_merged['cred_date']).dt.days

In [ ]:
# Categorizar os clientes em intervalo de tempo
bins = [0, 30, 60, 90, 180, float('inf')]
labels = ["0-30 dias", "31-60 dias", "61-90 dias", "90-179 dias", "180 dias ou mais"]
df_merged['categoria_tempo'] = pd.cut(df_merged['dias_apos_venda'], bins=bins, labels=labels, right=True)

# Agrupar e contar chamados
chamados_por_categoria = df_merged.groupby('categoria_tempo').size().reset_index(name='count')

# Criar coluna com percentual de cada categoria:
chamados_por_categoria['percentual'] = round((chamados_por_categoria['count'] / chamados_por_categoria['count'].sum()*100),1)

# Criar o gráfico com Plotly Express
fig = px.bar(chamados_por_categoria, x='categoria_tempo', y='percentual', title='Chamados por Categoria de Tempo desde a Venda', labels={'categoria_tempo': 'Tempo desde a venda', 'percentual': 'Número de Chamados (%)'})
fig.show()

In [ ]:
# Viz em formato de tabela

chamados_por_categoria.head()

,categoria_tempo,count,percentual
0,0-30 dias,174454,57.8
1,31-60 dias,53690,17.8
2,61-90 dias,27115,9.0
3,90-179 dias,28927,9.6
4,180 dias ou mais,17377,5.8


In [ ]:
# Estatistica descritiva

df_merged['dias_apos_venda'].describe()

count    326955.000000
mean         46.430001
std          73.101680
min        -231.000000
25%           8.000000
50%          20.000000
75%          54.000000
max         457.000000
Name: dias_apos_venda, dtype: float64

In [ ]:
# Boxplot para entender a distribuição

fig_2 = px.box(df_merged, y="dias_apos_venda")
fig_2.show()

In [ ]:
df_merged['accountid'].nunique()

31429

In [ ]:
df_merged.head()

,accountid,date_ref,channelid,waitingtime,missed,pesquisa_de_satisfa_o__c,assunto,Id,cred_date,shipping_address_city,shipping_address_state,max_machine,dias_apos_venda,categoria_tempo
0,0013j00002z0CeEAAU,2020-07-31,2.0,15.0,False,NaN,Aplicativo:Dúvidas funcionalidades App:Primeir...,0013j00002z0CeEAAU,2020-07-28,São Gonçalo,RJ,T3,3,0-30 dias
1,0013j00002z0CeEAAU,2020-07-31,2.0,15.0,False,NaN,Aplicativo:Dúvidas funcionalidades App:Primeir...,0013j00002z0CeEAAU,2020-07-28,São Gonçalo,RJ,T3,3,0-30 dias
2,0013j00002z0CeEAAU,2020-07-31,2.0,15.0,False,NaN,Aplicativo:Dúvidas funcionalidades App:Primeir...,0013j00002z0CeEAAU,2020-07-28,São Gonçalo,RJ,T3,3,0-30 dias
3,0013j00002z0CeEAAU,2020-07-31,2.0,15.0,False,NaN,Produto:S920:Ativação,0013j00002z0CeEAAU,2020-07-28,São Gonçalo,RJ,T3,3,0-30 dias
4,0013j00002z0CeEAAU,2020-07-31,2.0,15.0,False,NaN,Produto:S920:Ativação,0013j00002z0CeEAAU,2020-07-28,São Gonçalo,RJ,T3,3,0-30 dias


## Conclusão

Aproximadamente 58% dos tickets no período analisado são abertos por um cliente em seu primeiro mês após o credenciamento.

O gráfico na célula abaixo mostra o percentual do total dos chamados abertos, de acordo com intervalos de meses após o credenciamento.

A lógica para a resolução do problema foi a seguinte:



1.   Atrelar ao ticket a data de credenciamento do cliente que o abriu.
2.   Calcular a diferença da data de abertura para a data de credenciamento.
3.   Propor intervalos de tempo em meses e agrupar os tickets do mesmo intevalo
4.   Calcular a representação percentual da quantidade de chamados de cada grupo de intervalo em relação ao total de tickets abertos.



In [ ]:
# Categorizar os clientes em intervalo de tempo
bins = [0, 30, 60, 90, 180, float('inf')]
labels = ["0-30 dias", "31-60 dias", "61-90 dias", "90-179 dias", "180 dias ou mais"]
df_merged['categoria_tempo'] = pd.cut(df_merged['dias_apos_venda'], bins=bins, labels=labels, right=True)

# Agrupar e contar chamados
chamados_por_categoria = df_merged.groupby('categoria_tempo').size().reset_index(name='count')

# Criar coluna com percentual de cada categoria:
chamados_por_categoria['percentual'] = round((chamados_por_categoria['count'] / chamados_por_categoria['count'].sum()*100),1)

# Criar o gráfico com Plotly Express
fig = px.bar(chamados_por_categoria, x='categoria_tempo', y='percentual', title='Chamados por Categoria de Tempo desde a Venda', labels={'categoria_tempo': 'Tempo desde o credenciamento', 'percentual': 'Número de Chamados (%)'})
fig.show()

# Q3

Para conseguir melhorar a qualidade do atendimento ao cliente, foi proposto que a fila de espera para atendimento contemple prioridade para certos casos de atendimento. Proponha uma clusterização da base de clientes que julgue atender da melhor forma a necessidade apresentada.

## Desenvolvimento

In [ ]:
# Explorando os dados do assunto de chamado
assuntos = df_case['assunto'].unique()
print(assuntos)

['Aplicativo:Dúvidas funcionalidades App:Primeiro Acesso'
 'Produto:S920:Ativação' 'Produto:Cartão pré-pago:Dúvidas'
 'Logística:Envio de Cartão:Tracking' 'Pedido:Cancelamento:Outros'
 'Aplicativo:Dúvidas funcionalidades App:Redefinição de senha'
 'Aplicativo:Dúvidas funcionalidades App:Prazos e taxas'
 'Logística:OS:Confirmação/alteração de dados'
 'Aplicativo:Cartões:Redefinição de senha' 'Risco:Dúvidas bloqueio:'
 'Aplicativo:Dúvidas funcionalidades App:Saldo'
 'Logística:Envio de Cartão:Ineficiência'
 'Aplicativo:Dúvidas funcionalidades App:Link de Pagamento'
 'Produto:D195:Dúvida D195' 'Transação:Erro de transação:'
 'Produto:Pinpad:Problema Revertido'
 'Transferência de saldo:Conta bancária:'
 'Cadastro:Alteração de dados:Nome fantasia'
 'Cadastro:Alteração de dados:Outros'
 'Aplicativo:Dúvidas funcionalidades App:Transferência bancária'
 'Aplicativo:Cartões:Ativação' 'Produto:D195:Ativação'
 'Logística:Troca:Pinpad' 'Pedido:Confirmação do pedido:'
 'Logística:OS:Tracking de OS' 

In [ ]:
df_assunto = df_case.copy()

# Dividindo a coluna 'assunto' e criando três novas colunas no DataFrame
novas_colunas = df_assunto['assunto'].str.split(':', expand=True)
df_assunto['categoria'] = novas_colunas[0]
df_assunto['subcategoria'] = novas_colunas[1]
df_assunto['detalhe'] = novas_colunas[2]

### Verificar a quantidade de nulls nas feature:
feat_null_df_assunto = df_assunto.isnull().sum()
print("Valores nulos por feature:\n", feat_null_df_assunto)

Valores nulos por feature:
 accountid                       0
date_ref                        0
channelid                       0
waitingtime                     0
missed                          0
pesquisa_de_satisfa_o__c    65904
assunto                         0
Id                              0
categoria                       0
subcategoria                    0
detalhe                         0
dtype: int64


In [ ]:
df_assunto.head()

,accountid,date_ref,channelid,waitingtime,missed,pesquisa_de_satisfa_o__c,assunto,Id,categoria,subcategoria,detalhe
4,0013j00002z0CeEAAU,2020-07-31,2.0,15.0,False,NaN,Aplicativo:Dúvidas funcionalidades App:Primeir...,0013j00002z0CeEAAU,Aplicativo,Dúvidas funcionalidades App,Primeiro Acesso
5,0013j00002z0CeEAAU,2020-07-31,2.0,15.0,False,NaN,Produto:S920:Ativação,0013j00002z0CeEAAU,Produto,S920,Ativação
6,0013j00002z0CeEAAU,2020-09-23,2.0,1.0,False,NaN,Produto:Cartão pré-pago:Dúvidas,0013j00002z0CeEAAU,Produto,Cartão pré-pago,Dúvidas
7,0013j00002zQgldAAC,2020-08-29,2.0,6.0,False,NaN,Logística:Envio de Cartão:Tracking,0013j00002zQgldAAC,Logística,Envio de Cartão,Tracking
8,0013j00002zQgldAAC,2020-08-29,2.0,6.0,False,NaN,Pedido:Cancelamento:Outros,0013j00002zQgldAAC,Pedido,Cancelamento,Outros


In [ ]:
# Checando os valores únicos das novas features

df_assunto['categoria'].unique(), df_assunto['subcategoria'].unique(), df_assunto['detalhe'].unique()

(array(['Aplicativo', 'Produto', 'Logística', 'Pedido', 'Risco',
        'Transação', 'Transferência de saldo', 'Cadastro', 'Bandeiras',
        'Bandeira', 'Feedback do cliente', 'Outros', 'Telecom',
        'Incidente', '', 'Comunicados'], dtype=object),
 array(['Dúvidas funcionalidades App', 'S920', 'Cartão pré-pago',
        'Envio de Cartão', 'Cancelamento', 'OS', 'Cartões',
        'Dúvidas bloqueio', 'D195', 'Erro de transação', 'Pinpad',
        'Conta bancária', 'Alteração de dados', 'Troca',
        'Confirmação do pedido', 'Modelo de recebimento',
        'Dúvidas Bandeiras', 'Aplicativo', 'Promoções', '',
        'Consulta de transação', 'Atendimento', 'Envio Chip', 'Pagamentos',
        'Envio de Bobina', 'Desinstalação', 'Consulta de Dados', 'mPOS',
        'Máquina', 'Bloqueio', 'Simulação de vendas', 'Recredenciamento',
        'Instalação', 'Reativação de chip', 'Envio de Cartão pré-pago',
        'Problema', 'Time Out', 'Boleto', 'Envio de adesivo',
        'Cancelame

### Volume por categoria e subcategorias

In [ ]:
# Preparar os dados para o gráfico
data_for_plot = df_assunto.groupby(['categoria', 'subcategoria']).size().reset_index(name='contagem')

# Criar gráfico de barras empilhadas
fig = px.bar(data_for_plot,
             x='categoria',
             y='contagem',
             color='subcategoria',
             title='Distribuição de Subcategorias por Categoria Principal')

fig.show()

In [ ]:
# Agrupar por data e categoria, e contar os chamados
data_series = df_assunto.groupby(['date_ref', 'categoria']).size().reset_index(name='contagem')

# Criar o gráfico de série temporal
fig = px.line(data_series, x='date_ref', y='contagem', color='categoria',
              title='Série Temporal da Contagem das Categorias',
              labels={'date_ref': 'Data', 'Contagem': 'Número de Chamados'})

fig.show()


In [ ]:
# Extrair ano e mês da data
df_assunto_m = df_assunto.copy()
df_assunto_m['ano_mes'] = df_assunto_m['date_ref'].dt.to_period('M')

# Agrupar por ano_mes e Categoria Principal, e contar os chamados
data_series = df_assunto_m.groupby(['ano_mes', 'categoria']).size().reset_index(name='contagem')

# Ajustar o formato de 'ano_mes' para string para visualização
data_series['ano_mes'] = data_series['ano_mes'].dt.strftime('%Y-%m')

# Criar o gráfico de série temporal
fig = px.line(data_series, x='ano_mes', y='contagem', color='categoria',
              title='Série Temporal Mensal da Contagem das Categorias',
              labels={'ano_mes': 'Ano e Mês', 'Contagem': 'Número de Chamados'})

# Mostrar o gráfico
fig.show()


### Regras de associação
 Realizar a mineração de regras de associação através do algoritmo 'Apriori'. Esse algoritmo é utilizado para encontrar conjuntos de itens frequentes em bases de dados e, com base nesses conjuntos, gerar regras de associação. No contexto do case estas regras podem nos dizer algo como "Se um cliente abre um chamado da Categoria X, também tende a abrir um outro chamado da Categoria Y".

In [ ]:
data = df_assunto.copy()
data.head()

,accountid,date_ref,channelid,waitingtime,missed,pesquisa_de_satisfa_o__c,assunto,Id,categoria,subcategoria,detalhe
4,0013j00002z0CeEAAU,2020-07-31,2.0,15.0,False,NaN,Aplicativo:Dúvidas funcionalidades App:Primeir...,0013j00002z0CeEAAU,Aplicativo,Dúvidas funcionalidades App,Primeiro Acesso
5,0013j00002z0CeEAAU,2020-07-31,2.0,15.0,False,NaN,Produto:S920:Ativação,0013j00002z0CeEAAU,Produto,S920,Ativação
6,0013j00002z0CeEAAU,2020-09-23,2.0,1.0,False,NaN,Produto:Cartão pré-pago:Dúvidas,0013j00002z0CeEAAU,Produto,Cartão pré-pago,Dúvidas
7,0013j00002zQgldAAC,2020-08-29,2.0,6.0,False,NaN,Logística:Envio de Cartão:Tracking,0013j00002zQgldAAC,Logística,Envio de Cartão,Tracking
8,0013j00002zQgldAAC,2020-08-29,2.0,6.0,False,NaN,Pedido:Cancelamento:Outros,0013j00002zQgldAAC,Pedido,Cancelamento,Outros


In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Transformação dos dados para análise dos chamados com base em 'accountid' e 'assunto'
transacoes = data.groupby('accountid')['subcategoria'].apply(list)

# Instanciar o TransactionEncoder
te = TransactionEncoder()

# Ajustar e transformar os dados
te_ary = te.fit(transacoes).transform(transacoes)
df_transformed = pd.DataFrame(te_ary, columns=te.columns_)

# Aplicar o algoritmo Apriori
frequent_itemsets = apriori(df_transformed, min_support=0.01, use_colnames=True)

# Gerar as regras de associação
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)

In [ ]:
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

                            antecedents                        consequents  \
0                                    ()                  (Cartão pré-pago)   
1                                    ()      (Dúvidas funcionalidades App)   
2                                    ()                               (OS)   
3                  (Alteração de dados)                  (Cartão pré-pago)   
4                     (Cartão pré-pago)               (Alteração de dados)   
..                                  ...                                ...   
93                    (Cartão pré-pago)  (Dúvidas funcionalidades App, OS)   
94    (OS, Dúvidas funcionalidades App)                             (D195)   
95  (Dúvidas funcionalidades App, D195)                               (OS)   
96                           (OS, D195)      (Dúvidas funcionalidades App)   
97                               (D195)  (OS, Dúvidas funcionalidades App)   

     support  confidence      lift  
0   0.010691    0.205128  

In [ ]:
# Filtrar regras com lift > 1
filtered_rules = rules[rules['lift'] > 1]

# Converter frozensets em strings
filtered_rules['antecedents'] = filtered_rules['antecedents'].apply(lambda x: ', '.join(list(x)))
filtered_rules['consequents'] = filtered_rules['consequents'].apply(lambda x: ', '.join(list(x)))

# Criar um gráfico de dispersão com Plotly Express
fig = px.scatter(filtered_rules, x='support', y='confidence', text=filtered_rules.index, hover_data=['antecedents', 'consequents', 'lift'])

# Config. das legendas
fig.update_traces(textposition='top center')
fig.update_layout(title='Regras de Associação com Lift > 1', xaxis_title='Suporte', yaxis_title='Confiança')
fig.show()

/tmp/ipykernel_16731/1430609614.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_16731/1430609614.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



O gráfico acima nos mostra duas características para cada associação com lift > 1, que são associações em que a aparição conjunta entre duas 'subcategorias' associadas é maior do que suas aparições de forma independente.

Característica 1 - Confiança:
A confiança nos diz, em valores %, a quantidade de vezes em que as subcontas associadas aparecem juntas.

Por ex: Associação 'Alteração de dados' e 'Cartão pré-pago' com confiança de 0.17. Esta regra indica que quando ocorre um chamado de "Alteração de dados", em aproximadamente 17% das vezes isso está associado a um chamado sobre "Cartão pré-pago".

Característica 2 - Suporte:
O suporte nos diz, também em valores %, o volume deste grupo associado no valor total.

Para o mesmo exemplo anterios:
Associação 'Alteração de dados' e 'Cartão pré-pago' com suporte de 0.025.
Isso nos diz que essa combinação representa 2.5% de todas as transações do dataset.

Uma possibilidade de priorização dos chamados é utilizar estas regras visando diminuir a abertura de um futuro chamado associado.

Por ex: Index 87, asssociação: 'Duvidas funcionalidades App' e 'Cartão pré-pago, Modelo de recebimento' tem confiança de 73%, ou seja, aparecem 73% das vezes juntas. A hipótese é que, se no primeiro chamado de 'Duvidas e funcionalidades app' o atendimento abordar também o tema de cartão pré-pago, podemos reduzir boa parte de futuros chamados deste segundo tema.

obs: esse algoritmo é relativamente simples e não leva em consideração fatores importantes, sendo apenas um exemplo do que pode ser feito para este tipo de solução. Evoluções como: levar em consideração a linha temporal, ou seja, qual 'subcategoria' foi aberto o chamado primeiro e se o problema foi ou não resolvido, são exemplo de melhorias podem aumentar significativamente a efetividade da solução.

### Cadeia de Markov

São modelos estatísticos que descrevem um sistema que transita de um estado para outro, com a probabilidade de cada estado dependendo apenas do estado anterior (propriedade de Markov). As Cadeias de Markov são usadas para modelar processos estocásticos (aleatórios) onde a dependência é entre estados sequenciais. No nosso contexto, a ideia dessa modelagem é ter a resposta de algo como: "A probabilidade de um cliente abrir um chamado da categoria 'Cancelamento' após abrir um chamado da categoria 'Dúvidas' é de 10%".

In [ ]:
df_markov_invert = df_assunto.copy()

In [ ]:
# Agrupando por accountid e criando a sequência de categorias
df_markov_invert['categoria_sequence'] = df_markov_invert.groupby('accountid')['subcategoria'].transform(lambda x: ','.join(x))

# Removendo duplicatas
df_markov_invert.drop_duplicates(subset=['accountid'], inplace=True)

In [ ]:
from collections import defaultdict

# Inicializando um novo dicionário para contar transições inversas
inverse_transition_counts = defaultdict(lambda: defaultdict(int))

# Contar as transições inversas
for seq in df_markov_invert['categoria_sequence']:
    categories = seq.split(',')
    for (cat1, cat2) in zip(categories[:-1], categories[1:]):
        inverse_transition_counts[cat2][cat1] += 1

# Convertendo contagens em probabilidades inversas
inverse_transition_probabilities = {cat2: {cat1: count / sum(cat1_counts.values())
                                           for cat1, count in cat1_counts.items()}
                                    for cat2, cat1_counts in inverse_transition_counts.items()}


In [ ]:
# Escolhendo a categoria de interesse
cat_especifica_invert = 'Cancelamento'

# Obter as probabilidades inversas para a categoria específica
prob_invert = inverse_transition_probabilities.get(cat_especifica_invert, {})

# Ordenando as probabilidades do maior para o menor
prob_ordenada_invert = sorted(prob_invert.items(), key=lambda x: x[1], reverse=True)

# Imprimir as probabilidades ordenadas
for cat, prob in prob_ordenada_invert:
    print(f"{cat}: {prob}")

OS: 0.2611607142857143
Dúvidas funcionalidades App: 0.15401785714285715
Alteração de dados: 0.060267857142857144
: 0.046875
Dúvidas Bandeiras: 0.044642857142857144
S920: 0.044642857142857144
Cartão pré-pago: 0.03571428571428571
D195: 0.03125
Troca: 0.026785714285714284
Cancelamento: 0.022321428571428572
Envio de Cartão: 0.020089285714285716
Conta bancária: 0.020089285714285716
Recredenciamento: 0.020089285714285716
Instalação: 0.017857142857142856
Confirmação do pedido: 0.017857142857142856
Promoções: 0.017857142857142856
Cartões: 0.017857142857142856
Modelo de recebimento: 0.015625
Envio Chip: 0.013392857142857142
Desinstalação: 0.013392857142857142
Erro de transação: 0.008928571428571428
Outros: 0.008928571428571428
Pagamentos: 0.008928571428571428
Problema: 0.008928571428571428
Pinpad: 0.008928571428571428
Logística: 0.006696428571428571
Inbound: 0.006696428571428571
Atendimento: 0.006696428571428571
Aplicativo: 0.004464285714285714
Envio de Bobina: 0.004464285714285714
Máquina: 0.0

## Conclusão





A partir da exploração sobre os chamados na seção 'Desenvolvimento', será proposta a seguinte clusterização dos atendimentos:


------------------------------------

**Prio 1** - Chamados das categoria 'Transação', 'Risco' e 'Pedido'.
1.    Transação/Risco: aqui o cliente está com problema no uso, afetando direta e drasticamente a experiência com o produto, com a empresa e afetando diretamente a receita da Ton e do cliente.
2.    Pedido: nesta categoria são chamados do primeiro contato com o cliente, a lógica é de aqui é a falta ou demora de atendimento pode afetar a relação futura e a credibilidade de resolução da Ton. Além disso, é a categoria que engloba a subcategoria de 'Cancelamento', ou seja, um assunto urgente que envolve insatisfação e perda de receita.

------------------------------------

**Prio 2** - Chamados das subcategorias: 'OS', 'Dúvidas funcionalidade App' e 'Alteração dos dados'.

No algoritmo Cadeia de Markov foi constatado que aproximadamente 50% dos chamados abertos para a subcategoria 'Cancelamento' são criadas depois da abertura de chamados para as categorias:

1.   'OS',
2.   'Dúvidas funcionalidade App'
3.   'Alteração dos dados'

Sendo assim, a fim de evitar o cancelamento, essas subcategorias serão priorizadas.

Além, no algoritmo 'apriori' na seção 'Regras de associação', vimos que a maioria das associações tem em sua composição a subcategoria 'Dúvidas funcionalidade App'.

Isso quer dizer que, mesmo que não seja uma dúvida que leve ao cancelamento, podemos aproveitar a oportunidade com uma abordagem que vise reduzir a abertura de novos chamados, diminuindo assim o número de tickets no futuro associados a 'Dúvidas funcionalidade App'.

------------------------------------

**Prio 3** - Demais categorias.

Todos os casos que não se enquadram na prio 1 ou 2 ficarão neste cluster.

------------------------------------

A ordenação interna dos cluster será feita por:

1.   Tipo de canal de contato: prioridade para telefone e chat.
2.   tempo de atendimento, seguindo a lógica first in, first out.

------------------------------------

Logicamente é possível desenvolver estes e outros algoritmos para tornar a fila de atendimento ainda melhor, além de agregar dados de outras fontes com potencial para agregar na satisfação do cliente.

 Dados qualitativos de discoveries com as partes envolvidas no processo também são insumos valiosos para desvendar pontos críticos da jornada.

 Contudo, acredito que a ideia geral do que imagino para este tipo de problema está ok para o case. Espero que possamos implementar no futuro :)


# Q4
Usando as tabelas disponibilizadas, faça um query que retorne o volume de chamados por semana dos últimos três meses para cada cluster de clientes proposto na questão 3.

## Desenvolvimento

In [ ]:
df_assunto['subcategoria'].unique()

array(['Dúvidas funcionalidades App', 'S920', 'Cartão pré-pago',
       'Envio de Cartão', 'Cancelamento', 'OS', 'Cartões',
       'Dúvidas bloqueio', 'D195', 'Erro de transação', 'Pinpad',
       'Conta bancária', 'Alteração de dados', 'Troca',
       'Confirmação do pedido', 'Modelo de recebimento',
       'Dúvidas Bandeiras', 'Aplicativo', 'Promoções', '',
       'Consulta de transação', 'Atendimento', 'Envio Chip', 'Pagamentos',
       'Envio de Bobina', 'Desinstalação', 'Consulta de Dados', 'mPOS',
       'Máquina', 'Bloqueio', 'Simulação de vendas', 'Recredenciamento',
       'Instalação', 'Reativação de chip', 'Envio de Cartão pré-pago',
       'Problema', 'Time Out', 'Boleto', 'Envio de adesivo',
       'Cancelamento de Venda', 'Máquina roubada / perdida', 'Outros',
       'Logística', 'Cartão', 'Inbound', 'Transferência Terceiros',
       'Gestão de equipamentos', 'Blacklist'], dtype=object)

In [ ]:
# Função para determinar o cluster:

def cluster(row):
    if row['categoria'] in ['Pedido', 'Transação', 'Risco']:
        return 1
    elif row['subcategoria'] in ['OS', 'Dúvidas funcionalidades App', 'Alteração de dados']:
        return 2
    else:
        return 3

# Aplicando a função:
df_assunto['cluster'] = df_assunto.apply(cluster, axis=1)

df_assunto.head()

,accountid,date_ref,channelid,waitingtime,missed,pesquisa_de_satisfa_o__c,assunto,Id,categoria,subcategoria,detalhe,cluster
4,0013j00002z0CeEAAU,2020-07-31,2.0,15.0,False,NaN,Aplicativo:Dúvidas funcionalidades App:Primeir...,0013j00002z0CeEAAU,Aplicativo,Dúvidas funcionalidades App,Primeiro Acesso,2
5,0013j00002z0CeEAAU,2020-07-31,2.0,15.0,False,NaN,Produto:S920:Ativação,0013j00002z0CeEAAU,Produto,S920,Ativação,3
6,0013j00002z0CeEAAU,2020-09-23,2.0,1.0,False,NaN,Produto:Cartão pré-pago:Dúvidas,0013j00002z0CeEAAU,Produto,Cartão pré-pago,Dúvidas,3
7,0013j00002zQgldAAC,2020-08-29,2.0,6.0,False,NaN,Logística:Envio de Cartão:Tracking,0013j00002zQgldAAC,Logística,Envio de Cartão,Tracking,3
8,0013j00002zQgldAAC,2020-08-29,2.0,6.0,False,NaN,Pedido:Cancelamento:Outros,0013j00002zQgldAAC,Pedido,Cancelamento,Outros,1


In [ ]:
# Criando uma cópia do dataframe:
df_query = df_assunto.copy()
df_query.head()

,accountid,date_ref,channelid,waitingtime,missed,pesquisa_de_satisfa_o__c,assunto,Id,categoria,subcategoria,detalhe,cluster
4,0013j00002z0CeEAAU,2020-07-31,2.0,15.0,False,NaN,Aplicativo:Dúvidas funcionalidades App:Primeir...,0013j00002z0CeEAAU,Aplicativo,Dúvidas funcionalidades App,Primeiro Acesso,2
5,0013j00002z0CeEAAU,2020-07-31,2.0,15.0,False,NaN,Produto:S920:Ativação,0013j00002z0CeEAAU,Produto,S920,Ativação,3
6,0013j00002z0CeEAAU,2020-09-23,2.0,1.0,False,NaN,Produto:Cartão pré-pago:Dúvidas,0013j00002z0CeEAAU,Produto,Cartão pré-pago,Dúvidas,3
7,0013j00002zQgldAAC,2020-08-29,2.0,6.0,False,NaN,Logística:Envio de Cartão:Tracking,0013j00002zQgldAAC,Logística,Envio de Cartão,Tracking,3
8,0013j00002zQgldAAC,2020-08-29,2.0,6.0,False,NaN,Pedido:Cancelamento:Outros,0013j00002zQgldAAC,Pedido,Cancelamento,Outros,1


## Conclusão

In [ ]:
import sqlite3

# Converter a coluna 'Data' para o formato de datetime
df_query['date_ref'] = pd.to_datetime(df_query['date_ref'])

# Criar uma conexão SQLite temporária
conn = sqlite3.connect(':memory:')

# Copiar o DataFrame filtrado para uma tabela SQL
df_query.to_sql('df_table', conn, index=False, if_exists='replace')

# Consulta SQL
query = """
SELECT
    cluster,
    strftime('%W', date_ref) AS semana,
    COUNT(*) AS contagem_chamados
FROM
    df_table
WHERE
    date_ref >= datetime('now', '-3 years')
GROUP BY
    cluster, semana
ORDER BY
    semana
"""

# Executar a consulta SQL e ler os resultados em um novo DataFrame
result = pd.read_sql_query(query, conn)

# Fechar a conexão SQLite
conn.close()

print(result)
# Irá retornar conjunto vazio pois as datas do case são de 2020 e estamos buscando na query os últimos 3 meses.
# Caso queira ver algum output, basta trocar na cláusula 'WHERE' a instrução date_ref >= datetime('now', '-3 years') para a condição desejada.
# Ex: date_ref >= '2020-05-01', data do chamado maior igual a primeiro de Maio de 2020

Empty DataFrame
Columns: [cluster, semana, contagem_chamados]
Index: []


# Q5

Para melhorar o direcionamento da área de Relacionamento com o Cliente, a gestão
precisa do insumo necessário para entender como se comportam nossos clientes e
assim melhorar a tomada de decisão. Apresente pelo menos 3 KPIs que julgue
relevantes para acompanhamento diário da operação do Relacionamento com o
Cliente.

## Conclusão

Levando em consideração os dados do case, vejo que as principais métricas para o acompanhamento diário da gestão são:

1. **Taxa de tickets abertos por cliente:**

Insight: Mostra a frequência com que os clientes abrem chamados.

Importância: Identificação de problemas recorrentes e melhoria do produto.

Dados: count dos chamados e 'accountid' distintos.

Obs: Assim como abordamos na Q1, retirar os clientes que deram churn é fundamental. Isso pode ser feito subtraindo os chamados da subcategoria 'Cancelamento' ou provavelmente a melhor prática seria fazer a ingestão de outra base mais precisa.

2. **Taxa de atendimento dos chamados:**

Insight: Identifica falhas no atendimento, indicando períodos de alta demanda ou falta de recursos.

Importância: Eficácia do atendimento.

Dados: 'missed' e contagem total dos chamados abertos.

3. **Tempo médio de espera:**

Insight: Reflete a eficiência do atendimento e a capacidade da equipe em responder rapidamente.

Importância: Eficiência operacional.

Dados: 'waitingtime' e count dos chamados.

4. **Distribuição dos canais de comunicação:**

Insight: Mostra as preferências dos clientes quanto aos canais de comunicação.

Importância: Otimização de recursos e experiência do cliente.

Dados: 'channelid' e count dos chamados.

5. **Quantidade de chamados de determinados assuntos:**

Insight: Identifica as principais questões e preocupações dos clientes. Para se adequar ao formato de KPI, seria interessante selecionar assuntos críticos dos chamados junto a diretoria (ex: Erro de transação) e medir a quantidade percentual ao longo do tempo destes assuntos específicos.

Importância: Medir os resultados de ações estratégicas.

Dados: 'assunto' e count dos chamados, o dado que falta seria a definição de quais são estes assuntos prios para a empresa.


6. **Quantidade de chamados por produto:**

Insight: Identifica se certos produtos estão associados a mais aberturas de chamados.

Importância: Melhorias de produto e estratégias de comunicação.

Dados: 'max_machine' e count de chamados.

7. **Tempo desde o Credenciamento até o Primeiro Chamado:**

Insight: Mostra quão rapidamente os clientes enfrentam problemas após a aquisição.

Importância: Correção de problemas no início do ciclo de vida do cliente e melhorias no onboarding.

Dados: 'cred_date' e 'date_ref' do primeiro chamado.

-----------------------------------------------------------------
Outros indicadores que podem gerar informações relevantes mas que não temos os dados nas tabelas do case. Estas sugestões podem ser desenvolvidas a partir de uma ingestão/construção de outras fontes de dados.

-----------------------------------------------------------------

8. **Taxa de satisfação do cliente:**

Insight: Fornece uma medida direta da satisfação do cliente com o atendimento recebido.

Importância: Qualidade do serviço e fidelidade do cliente.

Dados: Não temos o retorno da pesquisa de satisfação, apenas o campo 'pesquisa_de_satisfação_c' que indica se foi enviada a pesquisa ou não. É fundamental coletar o resultado da pesquisa e atrelar aos dados já existentes.

9. **Tempo médio de resolução:**

Insight: Indica a eficiência na resolução de problemas.

Importância: Satisfação do cliente e confiança na empresa.

Dados: Não temos os dados que dizem se o problema foi resolvido ou não, apenas se o chamado foi atendido. Integrar ou criar uma base para entender se o problema do chamado foi é fator crítico ao desenvolvimento da área.

10. **Taxa de churn de clientes credenciados:**

Insight: Revela a taxa de perda de clientes após interações de suporte.

Importância: Eficácia do atendimento na retenção de clientes.

Dados: 'accountid' e dados fieis de cancelamento/encerramento de contas, necessário integração de outras fontes.




In [ ]:
!pip freeze

absl-py==1.4.0
aiohttp==3.9.3
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
altair==4.2.2
annotated-types==0.6.0
anyio==3.7.1
appdirs==1.4.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array-record==0.5.0
arviz==0.15.1
astropy==5.3.4
asttokens==2.4.1
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.14.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.22.1
bigframes==0.20.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
bqplot==0.12.42
branca==0.7.1
build==1.0.3
CacheControl==0.14.0
cachetools==5.3.2
catalogue==2.0.10
certifi==2024.2.2
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.7
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpathlib==0.16.0
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.0
colorcet==3.0.1
colorlover==0.3.0
colour==0.1.5
comm==0.2.1
community==1.0.0b1
confection==0.1.4
cons==0.4.6
contextlib2==21.6.0
contourpy==1.2.0
cryptography==42.0.2
cufflinks==